# 8.1 텍스트 처리용 IMDb 영화 리뷰 데이터 준비 

**의견분석**(감성분석) : 광범위한 NLP 연구 분야중 인기 있는 하위 분야  

IMDb의 영화 리뷰 데이터셋 사용  
- 레이블 설명
    - 긍정 : 리뷰 별 여섯 개 이상 받은 영화
    - 부정 : 리뷰 별 여섯 개 미만 받은 영화

## 8.1.2 영화 리뷰 데이터셋을 더 간편한 형태로 전처리 

중첩된 for 반복문에서 aclImdb 디렉터리의 하위 디렉터리 train, test와 그 아래 pos와 neg 디렉터리에서 개별 텍스트 파일 읽기

In [1]:
import pyprind
import pandas as pd
import os

basepath = 'aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], 
                           ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:34


데이터 프레임을 섞고 csv파일로 저장

In [2]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [3]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [4]:
df.shape

(50000, 2)

# 8.2 BoW 모델 소개 

**BoW**(Bag-of-Word) 모델 아이디어 - 텍스트를 수치 특성 벡터로 표현  

1. 전체 문서에 대해 고유한 토큰, 예를 들어 단어로 이루어진 어휘 사전 생성
2. 특정 문서에 각 단어가 얼마나 자주 등장하는지 파악하여 문서의 특성 벡터 생성  

각 문서에 있는 고유한 단어는 BoW 어휘 사전에 있는 모든 단어의 일부분에 지나지 않아 특성 벡터는 대부분 0으로 채워짐  
이런 벡터를 **희소 행렬**이라고 부름

## 8.2.1 단어를 특성 벡터로 변환 

CountVectorizer의 fit_transform 메서드를 호출하여 BoW 모델의 어휘사전을 만들고 다음 세 문장을 희소한 특성 벡터로 변환:
1. The sun is shining
2. The weather is sweet
3. The sun is shining, the weather is sweet, and one and one is two

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


특성 벡터의 각 인덱스는 CountVectorizer의 어휘 사전 딕셔너리에 저장  
**tf(t,d)** : 문서 d에 등장한 단어 t의 횟수(단어 빈도)

In [6]:
# 만들어진 특성 벡터

print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


#### n-그램 모델

n-그램 개념 - ngram_range 매개변수를 사용하여 변환  

`the sun is shining`  
- 1-그램 : `the`, `sun`, `is`, `shining`  
- 2-그램 : `the sun`, `sun is`, `is shining` 

## 8.2.2 tf-idf를 사용하여 단어 적합성 평가 

텍스트 데이터를 분석할 때 클래스 레이블이 다른 문서에 같은 단어들이 나타나는 경우를 종종 보게 됩니다. 일반적으로 자주 등장하는 단어는 유용하거나 판별에 필요한 정보를 가지고 있지 않는다.  
tf-idf(term frequency-inverse document frequency) : 특성 벡터에서 자주 등장하는 단어의 가중치를 낮추는 기법  
tf-idf : `tf(t, d)`와 `역문서 빈도(inverse document frequency)`의 곱으로 정의

$\text{tf-idf}(t,d)=\text{tf (t,d)}\times \text{idf}(t,d)$

- $idf(t, d)$ : 역문서 빈도

$\text{idf}(t,d) = \text{log}\frac{n_d}{1+\text{df}(d, t)}$

- $n_d$ : 전체 문서 개수  
- $df(d, t)$ : 단어 t가 포함된 문서 d의 개수  
- 분모에 상수 1을 추가하는 것 훈련 샘플에 한 번도 등장하지 않는 단어가 있는 경우 분모가 0이 되지 않도록 보정 
- log는 문서 빈도 $df(d, t)$가 낮을 때 역문서 빈도 값이 너무 커지지 않도록 만듭니다.

사이킷런 라이브러리에는 `CountVectorizer` 클래스에서 만든 단어 빈도를 입력받아 tf-idf로 변환하는 `TfidfTransformer` 클래스가 구현되어 있음

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, 
                         norm='l2', 
                         smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs))
      .toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


수동으로 특성 벡터에 있는 각 단어의 tf-idf를 계산해 보면 `TfidfTransformer`가 앞서 정의한 표준 tf-idf 공식과 조금 다르게 계산한다는 것을 알 수 있다.  

- 사이킷런에 구현된 역문서 빈도 공식

    - $\text{idf} (t,d) = log\frac{1 + n_d}{1 + \text{df}(d, t)}$

- 사이킷런에서 계산하는 tf-idf 공식

    - $\text{tf-idf}(t,d) = \text{tf}(t,d) \times (\text{idf}(t,d)+1)$

일반적으로 tf-idf를 계산하기 전에 단어 빈도(tf)를 정규화하지만 `TfidfTransformer` 클래스는 tf-idf를 직접 정규화합니다.  
사이킷런의 `TfidfTransformer`는 기본적으로 L2 정규화를 적용 (norm=’l2’)  
정규화되지 않은 특성 벡터 v를 L2-노름으로 나누면 길이가 1인 벡터가 반환

$v_{\text{norm}} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v_{1}^{2} + v_{2}^{2} + \dots + v_{n}^{2}}} = \frac{v}{\big (\sum_{i=1}^{n} v_{i}^{2}\big)^\frac{1}{2}}$

TfidfTransformer의 작동 원리를 이해하기 위해 세 번째 문서에 있는 단어 ‘is'의 tf-idf를 예시로 들면,

- 세 번째 문서에서 단어 ‘is’의 단어 빈도는 3 (tf=3) 
- 이 단어는 세 개 문서에 모두 나타나기 때문에 문서 빈도가 3 (df=3)  

    - $\text{idf}("is", d3) = log \frac{1+3}{1+3} = 0$

- 이제 tf-idf를 계산하기 위해 역문서 빈도에 1을 더하고 단어 빈도를 곱하기

    - $\text{tf-idf}("is",d3)= 3 \times (0+1) = 3$

In [8]:
tf_is = 3
n_docs = 3
idf_is = np.log((n_docs+1) / (3+1))
tfidf_is = tf_is * (idf_is + 1)
print('tf-idf of term "is" = %.2f' % tfidf_is)

tf-idf of term "is" = 3.00


- tf-idf 계산 L2-정규화 진행:  

$\text{tfi-df}_{norm} = \frac{[3.39, 3.0, 3.39, 1.29, 1.29, 1.29, 2.0 , 1.69, 1.29]}{\sqrt{[3.39^2, 3.0^2, 3.39^2, 1.29^2, 1.29^2, 1.29^2, 2.0^2 , 1.69^2, 1.29^2]}}$

$=[0.5, 0.45, 0.5, 0.19, 0.19, 0.19, 0.3, 0.25, 0.19]$

$\Rightarrow \text{tfi-df}_{norm}("is", d3) = 0.45$

In [9]:
tfidf = TfidfTransformer(use_idf=True, norm=None, smooth_idf=True)
raw_tfidf = tfidf.fit_transform(count.fit_transform(docs)).toarray()[-1]
raw_tfidf 

array([3.39, 3.  , 3.39, 1.29, 1.29, 1.29, 2.  , 1.69, 1.29])

In [10]:
l2_tfidf = raw_tfidf / np.sqrt(np.sum(raw_tfidf**2))
l2_tfidf

array([0.5 , 0.45, 0.5 , 0.19, 0.19, 0.19, 0.3 , 0.25, 0.19])

다음 절로 넘어가 이 개념을 영화 리뷰 데이터셋에 적용

## 8.2.3 텍스트 데이터 정제 

In [11]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

HTML 마크업과 구두점, 글자가 아닌 문자가 포함되어 있다.  
감성 분석에 유연한 `:)`같은 이모티콘 문자를 제외하고 모든 구두점 기호 삭제

In [12]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

In [13]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [14]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

In [15]:
df['review'] = df['review'].apply(preprocessor)

## 8.2.4 문서를 토큰으로 나누기 

In [16]:
# 어간 추출 사용법
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.split()


def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [17]:
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [18]:
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

**불용어**(stop-word) : 모든 종류의 텍스에 아주 흔하게 등장하는 단어 -> 문서의 종류를 구별하는데 큰 의미가 없음

In [19]:
# NLTK 라이브러리에서 제공하는 179개의 불용어 사용
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/junghyuntaek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
# 불용어 집합을 내려받은 후 다음과 같이 영어의 불용어를 불러들여 적용 가능
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:]
if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

# 8.3 문서 분류를 위한 로지스틱 회귀 모델 훈련 

GridSearch 속도를 높이기 위해 HTML과 구두점을 삭제

In [21]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0, solver='liblinear'))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           n_jobs=-1)

**`n_jobs` 매개변수에 대하여**

컴퓨터에 있는 모든 CPU 코어를 사용해 그리드 서치의 속도를 높이려면 (`n_jobs=1` 대신) `n_jobs=-1`로 지정하는 것이 좋다  
일부 시스템에서는 멀티프로세싱을 위해 `n_jobs=-1`로 지정할 때 `tokenizer` 와 `tokenizer_porter` 함수의 직렬화에 문제가 발생할 수 있다.  
이런 경우 `[tokenizer, tokenizer_porter]`를 `[str.split]`로 바꾸어 문제를 해결할 수 있지만 `str.split`로 바꾸면 어간 추출 불가능

**코드 실행 시간에 대하여**

다음 코드 셀을 실행하면 시스템에 따라 **30~60분 정도 걸릴 수 있다**  
매개변수 그리드에서 정의한 대로 ( 2 X 2 X 2 X 3 X 5 ) + ( 2 X 2 X 2 X 3 X 5 ) = 240개의 모델을 훈련하기 때문

**코랩을 사용할 경우에도 CPU 코어가 많지 않기 때문에 실행 시간이 오래 걸릴 수 있다.**

너무 오래 기다리기 어렵다면 데이터셋의 훈련 샘플의 수를 다음처럼 줄이기

```
    X_train = df.loc[:2500, 'review'].values
    y_train = df.loc[:2500, 'sentiment'].values
```
    
훈련 세트 크기를 줄이는 것은 모델 성능을 감소  
그리드에 지정한 매개변수를 삭제하면 훈련한 모델 수를 줄이기 가능

```
    param_grid = [{'vect__ngram_range': [(1, 1)],
                   'vect__stop_words': [stop, None],
                   'vect__tokenizer': [tokenizer],
                   'clf__penalty': ['l1', 'l2'],
                   'clf__C': [1.0, 10.0]},
                  ]
```

In [23]:
X_train = df.loc[:2500, 'review'].values
y_train = df.loc[:2500, 'sentiment'].values

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

In [24]:
gs_lr_tfidf.fit(X_train, y_train)

/Users/junghyuntaek/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/Users/junghyuntaek/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/Users/junghyuntaek/opt/anaconda3/

/Users/junghyuntaek/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/Users/junghyuntaek/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/Users/junghyuntaek/opt/anaconda3/

/Users/junghyuntaek/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/Users/junghyuntaek/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/Users/junghyuntaek/opt/anaconda3/

/Users/junghyuntaek/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0,
                                                           solver='liblinear'))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've...
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
 

In [25]:
print('최적의 매개변수 조합: %s ' % gs_lr_tfidf.best_params_)
print('CV 정확도: %.3f' % gs_lr_tfidf.best_score_)

최적의 매개변수 조합: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few'

In [26]:
clf = gs_lr_tfidf.best_estimator_
print('테스트 정확도: %.3f' % clf.score(X_test, y_test))

테스트 정확도: 0.856


# 8.4 대용량 데이터 처리: 온라인 알고리즘과 외부 메모리 학습 

In [27]:
import numpy as np
import re
from nltk.corpus import stopwords


# `stop` 객체를 앞에서 정의했지만 이전 코드를 실행하지 않고
# 편의상 여기에서부터 코드를 실행하기 위해 다시 만듭니다.
stop = stopwords.words('english')


def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized


def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # 헤더 넘기기
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [28]:
next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [29]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [30]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier


vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

In [31]:
from distutils.version import LooseVersion as Version

clf = SGDClassifier(loss='log', random_state=1)

doc_stream = stream_docs(path='movie_data.csv')

In [32]:
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:19


In [33]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('정확도: %.3f' % clf.score(X_test, y_test))

정확도: 0.868


In [34]:
clf = clf.partial_fit(X_test, y_test)

# 8.5 잠재 디리클레 할당을 사용한 토픽 모델링 

## 8.5.1 LDA를 사용한 텍스트 문서 분해 

## 8.5.2 사이킷런의 LDA

In [36]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [37]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english',
                        max_df=.1,
                        max_features=5000)
X = count.fit_transform(df['review'].values)

In [38]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10,
                                random_state=123,
                                learning_method='batch')
X_topics = lda.fit_transform(X)

In [39]:
lda.components_.shape

(10, 5000)

In [41]:
n_top_words = 5
feature_names = count.get_feature_names()
for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i]
                    for i in topic.argsort()\
                        [:-n_top_words - 1:-1]]))

Topic 1:
worst minutes awful script stupid
Topic 2:
family mother father children girl
Topic 3:
american war dvd music tv
Topic 4:
human audience cinema art sense
Topic 5:
police guy car dead murder
Topic 6:
horror house sex girl woman
Topic 7:
role performance comedy actor performances
Topic 8:
series episode war episodes tv
Topic 9:
book version original read novel
Topic 10:
action fight guy guys cool


각 토픽에서 가장 중요한 단어 다섯 개를 기반으로 LDA가 다음 토픽을 구별했다고 추측

1.	대체적으로 형편없는 영화(실제 토픽 카테고리가 되지 못함)
2.	가족 영화
3.	전쟁 영화
4.	예술 영화
5.	범죄 영화
6.	공포 영화
7.	코미디 영화
8.	TV 쇼와 관련된 영화
9.	소설을 원작으로 한 영화
10.	액션 영화

카테고리가 잘 선택됐는지 확인하기 위해 공포 영화 카테고리에서 3개 영화의 리뷰를 출력 (공포 영화는 카테고리 6이므로 인덱스는 5)

In [42]:
horror = X_topics[:, 5].argsort()[::-1]

for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\n공포 영화 #%d:' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], '...')


공포 영화 #1:
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...

공포 영화 #2:
Okay, what the hell kind of TRASH have I been watching now? "The Witches' Mountain" has got to be one of the most incoherent and insane Spanish exploitation flicks ever and yet, at the same time, it's also strangely compelling. There's absolutely nothing that makes sense here and I even doubt there  ...

공포 영화 #3:
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...


앞 코드에서 공포 영화 카테고리 중 최상위 3개의 리뷰에서 300자씩 출력  
정확히 어떤 영화에 속한 리뷰인지는 모르지만 공포 영화의 리뷰임을 알 수 있다(하지만 영화 #2는 1번 카테고리에 속하는 것처럼 보이기도 한다)